# Project 2: Moneyball
### by Jerry Catron


In [ ]:
Inspired by the 2011 film featuring my uglier doppelganger Brad Pitt, this report will analyze how efficient Major League Baseball teams have been historically at spending money and getting wins in return.

We will be using a database curated by Sean Lahman that has data on the teams, players, statistics, salaries, etc. Currently we are only analyzing the data through the 2014 season.

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('lahman2014.sqlite')